In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os
import asyncio
from typing import List
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

load_dotenv()

system_prompt = """You are an AI assistant for a search and rescue application. Your primary role is to support drone operators in mapping hazards and plotting safe routes.

Key capabilities:
1. Hazard reporting: You can analyze drone data and report potential hazards to the frontend interface.
2. Route planning: Based on the id of the personr in need of rescue and the list of hazards to avoid, you can suggest optimal routes for rescue teams.

Your responses should be clear, concise, and focused on providing actionable information to the drone operators. Prioritize safety and efficiency in all recommendations. When providing information or suggestions, always consider the urgency of search and rescue operations.

Remember, your guidance directly impacts the safety of both rescue teams and those in need of assistance. Maintain a professional and supportive tone at all times."""

@tool
async def display_hazards(hazards: List[str]):
    """Display hazards on the map.

    Args:
        hazards: List of types of hazards to display on the map. One of the following: "all", "person", "fire", "tree", "power", "follows". Default is "all". If user says do not display any hazards, set hazards to empty list.
    """
    print(hazards)
    
    return "Sent hazards to the frontend."

@tool
async def plan_route(id: str, hazards: List[str]):
    """Plan a route to help people avoid hazards. You only need the id of the person in need of rescue and the list of hazards to avoid.
    
    
    Args:
        id: The ID of the person in need of rescue.
        hazards: List of types of hazards to avoid. List of one or more of the following: "all", "person", "fire", "tree", "power", "flood". Default is empty list to signify no hazards avoided.
    """
    print(id, hazards)
    return "Route has been planned, and sent to the frontend."
    

tools = [display_hazards, plan_route]
memory = MemorySaver()

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", api_key=os.getenv("GEMINI_API_KEY"), temperature=0.0)
agent_executor = create_react_agent(llm, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}


async for event in agent_executor.astream_events(
    {"messages": [
        SystemMessage(content=system_prompt),
        HumanMessage(content="Can you display the fires in the area?")]}, config,version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")


Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


{'name': 'display_hazards', 'description': 'Display hazards on the map.\n\n    Args:\n        hazards: List of types of hazards to display on the map. One of the following: "all", "person", "fire", "tree", "power", "follows". Default is "all". If user says do not display any hazards, set hazards to empty list.', 'parameters': {'type_': 6, 'description': 'Display hazards on the map.\n\nArgs:\n    hazards: List of types of hazards to display on the map. One of the following: "all", "person", "fire", "tree", "power", "follows". Default is "all". If user says do not display any hazards, set hazards to empty list.', 'properties': {'hazards': {'type_': 5, 'items': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['hazards'], 'format_': '', 'nullab

In [16]:
async for event in agent_executor.astream_events({
    "messages":[
        HumanMessage(content="Can you display route me to the person with id abc and avoid fires and floods?")]}, config,version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")


{'name': 'display_hazards', 'description': 'Display hazards on the map.\n\n    Args:\n        hazards: List of types of hazards to display on the map. One of the following: "all", "person", "fire", "tree", "power", "follows". Default is "all". If user says do not display any hazards, set hazards to empty list.', 'parameters': {'type_': 6, 'description': 'Display hazards on the map.\n\nArgs:\n    hazards: List of types of hazards to display on the map. One of the following: "all", "person", "fire", "tree", "power", "follows". Default is "all". If user says do not display any hazards, set hazards to empty list.', 'properties': {'hazards': {'type_': 5, 'items': {'type_': 1, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}}, 'required': ['hazards'], 'format_': '', 'nullab